In [1]:
%matplotlib inline

In [3]:
# add repo path
import sys
sys.path.append('/home/gautam_sisodia/gen-purpose-repo/')

In [8]:
import gym
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from rl import frozenlake as fl

In [9]:
env = gym.make('FrozenLake-v0')
env.reset()
env.render()

[2017-10-08 14:40:16,952] Making new env: FrozenLake-v0



SFFF
FHFH
FFFH
HFFG


## General Stats
Gathering some general statistics about states from random actions

In [10]:
state_dict = False

In [13]:
state_dict, rand_game_df = fl.run_n_games(20000, fl.rand_strategy, state_dict)

  0%|          | 0/20000 [00:00<?, ?it/s]


NameError: name 'env' is not defined